In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#Helper function that is used to concatenate census data return
def create_or_append_df(df, summary_df):
    if df.empty:
        df = summary_df.copy()
    else:
        df = pd.concat([df, summary_df])
    return df



service_url = 'https://maps.trpa.org/server/rest/services/Transportation/MapServer/7'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
all_data = pd.DataFrame([feature.attributes for feature in feature_list])


In [3]:
download_years = [
    '2014-15', '2015-16','2016-17','2017-18','2018-19','2019-20','2020-21','2021-22','2022-23'
]

base_url = 'https://dq.cde.ca.gov/dataquest/dqcensus/EnrGrdLevels.aspx?cds=0961903&agglevel=district&year='

combined_data = pd.DataFrame()

for year in download_years:
    url = base_url+year
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': "ContentPlaceHolder1_grdEnrlPct"})
    df = pd.read_html(str(table))[0]
    df['school_year']=year
    combined_data = create_or_append_df(combined_data, df)
    
    


In [4]:
combined_data.to_excel("School_Enrollment.xlsx")

In [6]:
combined_data_grouped = combined_data.groupby(["school_year"])["Total"].sum()